# Google Apple contact tracing

* [source](https://covid19-static.cdn-apple.com/applications/covid19/current/static/contact-tracing/pdf/ContactTracing-CryptographySpecification.pdf)
* Techniques used: 
 1. Hash chains, 
 2. TTP, 
 3. set intersection
* There are privacy risks involved. The cryptography specification mentions a few but there are others too. For instance, the set intersection may reveal identities.

Bluetooth keeps advertising information. You can use the advertisment broadcast to send an identity of a user. Apple and Google have proposed using BTLE broadcasts for COVID-19 tracking in a privacy preserving way.

Method proposed is based on three keys: 1) Tracing key, 2) Daily Tracing key, and 3) Rolling Proximity Identifier. 

1. Tracing key, $tk \leftarrow CNRG(256)$, is a once produced 256-bit private master key. 
2. A once per day derived Daily tracing key as follows: $dtk_i \leftarrow truncate(SHA256(tk,(UTF8("CT-DTK")|| D_i)),16)$, where $D_i$ is the day number in unix time
3. 144 rolling proxy keys per day derived from $dtk_i$ as follows: $RPI_{i,j} = truncate(SHA256(dkt_i, (UTF8("CT-RPI") || TIN_j) ),16)$, where $TIN_j \in [0,143]$
4. The rolling proxy keys are broadcasted using BTLE
5. If Alice is infected with COVID, a subset of the daily keys called diagnosis keys are published by TTP Trent
6. Bob can download the list of diagnosis keys and generate all the rolling proxies
7. Bob checks for set intersection between list from 6 and any proxy key heard in 4.

In [2]:
from Crypto.Util import number
import time
import datetime
import numpy as np
import random
from Crypto.Hash import SHA256 # you should use something else as explained https://security.stackexchange.com/questions/133239/what-is-the-specific-reason-to-prefer-bcrypt-or-pbkdf2-over-sha256-crypt-in-pass

In [3]:
def daily_key(tracing_key, day):
    '''
    Takes as input the tracing key (a 256 bit random non-negative integer) and day in datetime.date format
    Returns the daily key as SHA-256 hexdigest
    '''
    assert type(day) == datetime.date, 'Input format for day must be datetime.date'
    day = date_to_unix_time([day])[0]
    tracing_key = str(tracing_key).encode('utf-8')
    key_daily = SHA256.new()
    key_daily.update(tracing_key)
    key_daily.update(str('CT-DTK').encode('utf-8') + day.to_bytes(4, 'little'))
    return key_daily.hexdigest()[:16]

def rolling_proxy_identifier(daily_key, n):
    '''
    Takes as input the tracing key (a 256 bit random non-negative integer) and time interval integer \in [0,143]
    Returns a 16-byte rolling proximity identifier from a SHA-256 hexdigest
    ''' 
    assert n >= 0 and 144 > n, 'TINj is out of range [0,143]'
    key_proxy = SHA256.new()
    key_proxy.update(daily_key.encode('utf-8'))
    key_proxy.update(str('CT-RPI').encode('utf-8') + str(n).encode('utf-8'))
    return key_proxy.hexdigest()[:16]

def time_interval_number():
    '''
    Returns the current TINj for the day when BTLE advert is sent
    '''
    x = time.time()
    y = x - int(x/(60*60*24)) * (60*60*24) # seconds since start of day number
    TINj = int(y / (60*10))
    return TINj

def diagnosis_keys(tracing_key, diagnosis_day, n):
    '''
    Takes as input an positive integer for the number of days where user could have been infected
    Returns daily keys for that range together with rolling_proxy_identifiers
    '''
    assert type(diagnosis_day) == datetime.date, 'Input format must be datetime.date'
    daily_keys = []
    for i in range(n):
        daily_keys.append(daily_key(tracing_key, day = (diagnosis_day - datetime.timedelta(days=i))))
    return daily_keys
    
def diagnosis_rolling_proxy_identifiers(daily_keys):
    '''
    Takes as input array of diagnosis keys
    Outputs array of rolling proxies
    '''
    assert type(daily_keys) == list, 'Input must be list of daily keys'
    RPIij = []
    for a in daily_keys:
        for i in range(144):
            RPIij.append(rolling_proxy_identifier(a, i))
    return RPIij

def date_to_unix_time(a):
    '''
    Input date as array with elements in format datetime.date
    Outputs unix times
    '''
    assert type(a) == list, 'Input must be list'
    assert all(isinstance(x, datetime.date) for x in a), 'Elements in list must be datetime.date'
    unix_times = []
    for i in a:
        unix_times.append(int(time.mktime(i.timetuple()))) # Converts string to unix time
    return sorted(unix_times)

def dates_in_public(a='2020-01-01', b=datetime.date.today(), n=25):
    '''
    Input dates range [a = yyyy-mm-dd_0, b = yyyy-mm-dd_1] and number of dates in public n
    Outputs random dates in unix time
    '''
    a = datetime.datetime.strptime(str(a), '%Y-%m-%d').date()
    b = datetime.datetime.strptime(str(b), '%Y-%m-%d').date()
    date_delta = b - a
    public_visits = [a + datetime.timedelta(days=i) for i in random.sample(range(date_delta.days + 1), n)]
    return sorted(public_visits)

In [11]:
# The tracing keys
alice_key_tracing = number.getRandomNBitInteger(256) # stored on Alice device

# Dates when Alice was outside
alice_dates_in_public = dates_in_public(n=2)

# The daily keys stored on Alice device for the spot visits
alice_daily_keys_in_public = []
for i in alice_dates_in_public:
    alice_daily_keys_in_public.append(daily_key(alice_key_tracing, i))
    
# Alice broadcasts rolling proxy ids
alice_rolling_proxy_broadcasted = {}
for i in alice_daily_keys_in_public:
    alice_rolling_proxy_broadcasted[i] = [rolling_proxy_identifier(i, a) for a in random.sample(range(144),5)] # We assume Alice broadcasts 5 proxy ids per day

# Bob may hear some broadcasts
bob_heard_alice = []
[bob_heard_alice.extend(i[1]) for i in alice_rolling_proxy_broadcasted.items()]
bob_heard_alice = random.sample(bob_heard_alice, random.randrange(0,len(bob_heard_alice)))

# Alice tests positive for COVID-19 and publishes her diagnosis keys to Trent
alice_diagnosis_keys = []
for i in alice_dates_in_public:
    alice_diagnosis_keys.extend(diagnosis_keys(alice_key_tracing, i, 14)) # 14 days is often used for COVID-19 contact window

# Bob gets list of diagnosis keys from Trent and generates list of proxy ids to check against
bob_proxy_ids_to_check = diagnosis_rolling_proxy_identifiers(alice_diagnosis_keys)
bob_heard_proxies = np.intersect1d(bob_proxy_ids_to_check, bob_heard_alice)
assert (len(bob_heard_proxies) != 0) == (len(bob_heard_alice) !=0), 'Something wrong with intersection'

print('Bob computes intersection:', bob_heard_proxies)
print('Bob should get tested') if len(bob_heard_proxies) != 0 else print('Bob does not need to get tested')

Bob computes intersection: []
Bob does not need to get tested
